^C
Note: you may need to restart the kernel to use updated packages.


^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy
import rioxarray

air50s = rioxarray.open_rasterio("E:/Google Drive/GIS/utica/MyProject4A550500171317_ref.tif")

ModuleNotFoundError: No module named 'rioxarray'